<a href="https://colab.research.google.com/github/Namyalg/Topic-Modeling/blob/master/COVID_LSA_Finalwork.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import sklearn
# Import all of the scikit learn stuff
from __future__ import print_function
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import Normalizer
from sklearn import metrics
from sklearn.cluster import KMeans, MiniBatchKMeans
import pandas as pd
import warnings
# Suppress warnings from pandas library
warnings.filterwarnings("ignore", category=DeprecationWarning,
module="pandas", lineno=570)
import numpy

In [2]:
import nltk
nltk.download("stopwords")
nltk.download("wordnet")
nltk.download("punkt")
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
stop = set(stopwords.words('english'))
from nltk.stem.porter import PorterStemmer
from gensim import corpora, models
import gensim
import string
exclude = set(string.punctuation)
lemma = WordNetLemmatizer()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
#tf.idf is very simple to understand 
#term frequency means how many times a term appears in a given document meaning, frequency of a word in a document,
#A document can mean anything from a simple sentence to a paragraph

#tf = number of times a word appears / Total number of words , in every document

#idf = inverse document frequency
#this will be to give words that are not so frequent a higher weight.

#so idf is calculated as the number of documents = log(N/ dfi)

#N is the total number of documents and the dfi stands for the number of documents a given term i appears in 

In [4]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfTransformer
dataset = [
    "I enjoy reading about Machine Learning and Machine Learning is my PhD subject",
    "I would enjoy a walk in the park",
    "I was reading in the library"
]

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
pd.set_option("display.max_colwidth", 200)

In [6]:
import csv
from csv import reader
tit = []
with open("/content/COVID-19_title.csv", 'r') as read_obj:
    # pass the file object to reader() to get the reader object
    csv_reader = reader(read_obj)
    # Iterate over each row in the csv using reader object
    for row in csv_reader:
        # row variable is a list that represents a row in csv
        tit.append(row)

In [7]:
print(tit[:5])

[['clinical features culture-proven mycoplasma pneumoniae infections king abdulaziz university hospital jeddah saudi arabia'], ['nitric oxide pro-inflammatory mediator lung diseas'], ['surfactant protein-d pulmonary host defens'], ['role endothelin-1 lung diseas'], ['gene expression epithelial cells response pneumovirus infect']]


In [8]:
val = []
for i in tit:
  val.append(i[0])

In [9]:
val = val[:1000]
print(val[:10])

['clinical features culture-proven mycoplasma pneumoniae infections king abdulaziz university hospital jeddah saudi arabia', 'nitric oxide pro-inflammatory mediator lung diseas', 'surfactant protein-d pulmonary host defens', 'role endothelin-1 lung diseas', 'gene expression epithelial cells response pneumovirus infect', 'sequence requirements strand transfer nidovirus discontinuous subgenomic synthesi', 'debate transfusing normal haemoglobin levels improve outcom', '21st international symposium intensive care emergency medicine brussels belgium 20-23 march 2001', 'heme oxygenase-1 carbon monoxide pulmonary medicin', 'technical description rods real-time public health surveillance system']


In [36]:
tokenizer = RegexpTokenizer(r'\w+')

# create English stop words list
#en_stop = set(stopwords.words('english'))
# Create p_stemmer of class PorterStemmer
p_stemmer = PorterStemmer()
    
doc_set = val
# list for tokenized documents in loop
texts = []

# loop through document list
for i in doc_set:
    
    # clean and tokenize document string
    raw = i.lower()
    tokens = tokenizer.tokenize(raw)
    texts.append(tokens)

# turn our tokenized documents into a id <-> term dictionary
dictionary = corpora.Dictionary(texts)
    
# convert tokenized documents into a document-term matrix
doc_term_matrix = [dictionary.doc2bow(text) for text in texts]

In [33]:
print(doc_term_matrix)

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1)], [(14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1)], [(21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1)], [(14, 1), (16, 1), (27, 1), (28, 1), (29, 1)], [(30, 1), (31, 1), (32, 1), (33, 1), (34, 1), (35, 1), (36, 1)], [(37, 1), (38, 1), (39, 1), (40, 1), (41, 1), (42, 1), (43, 1), (44, 1)], [(45, 1), (46, 1), (47, 1), (48, 1), (49, 1), (50, 1), (51, 1)], [(52, 1), (53, 1), (54, 1), (55, 1), (56, 1), (57, 1), (58, 1), (59, 1), (60, 1), (61, 1), (62, 1), (63, 1), (64, 1)], [(25, 1), (27, 1), (65, 1), (66, 1), (67, 1), (68, 1), (69, 1)], [(70, 1), (71, 1), (72, 1), (73, 1), (74, 1), (75, 1), (76, 1), (77, 1), (78, 1)], [(79, 1), (80, 1), (81, 1), (82, 1), (83, 1), (84, 1), (85, 1)], [(86, 1), (87, 1), (88, 1), (89, 1), (90, 1), (91, 1), (92, 1)], [(24, 1), (93, 1), (94, 1), (95, 1), (96, 1)], [(92, 1), (97, 1), (98, 1), (99, 1), (100, 1), (101, 1), (102, 1)

In [39]:
lsamodel = LsiModel(doc_term_matrix, num_topics=20, id2word = dictionary,decay=1.0, chunksize=20000)  # train model
for i in (lsamodel.print_topics(num_topics=20, num_words=6)):
  print(i)

TypeError: ignored

In [29]:
coherencemodel = CoherenceModel(model=lsamodel, texts=texts, dictionary=dictionary, coherence='c_v')
print(coherencemodel.get_coherence())

0.346945932301464


In [ ]:
from gensim import corpora, models
tfidf = models.TfidfModel(corpus)
corpus_tfidf = tfidf[corpus]


In [12]:
vectorizer = TfidfVectorizer()
vectors = vectorizer.fit_transform(val[:10000])
feature_names = vectorizer.get_feature_names()
dense = vectors.todense()
denselist = dense.tolist()
df = pd.DataFrame(denselist, columns=feature_names)

In [11]:
df.head()

,000,0201,07,11,1101,11th,12,13,14,15,18,1918,1919,1920,1975,1985,1995,1997,1999,1beta,1h,1β,20,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,21,21st,23,2336,24,...,wild,wildlife,willebrand,willingness,winner,winter,wireless,with,within,without,women,woodchuck,word,work,workers,working,world,wrong,xenografts,xiao,xmrv,y271,year,years,yeast,yersinia,yields,yolk,york,young,zanamivir,zealand,zipper,zone,zoonosi,zoonosis,zoonotic,αβ,β1,κb
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [21]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(val)
print(vectorizer.get_feature_names())

['00', '000', '0007', '00072', '00073', '0008', '001', '00125', '00135', '00140', '00181', '002', '00284', '002_05_2015', '0035', '0036', '004', '00464', '00467', '00483', '005', '0054', '00547659', '0055', '007', '00886', '008979', '01', '010', '0100', '0101', '0105', '0106', '012', '01220r1', '013', '014', '014053', '015', '017', '0175', '0176', '0177', '0184', '019', '0199688931', '02', '020', '0201', '021', '0212', '022', '0226626444', '02442r2', '025', '026', '0268355520930884', '027', '028', '03', '030', '0300256390', '0301', '03012', '03165', '03253', '03264', '033', '034', '0344', '035', '0354', '0358', '036', '0363', '0364', '037', '0372', '038', '039', '039229', '039230', '039233', '04', '0400', '0401', '0408', '0412', '0414', '042', '0420', '043', '04305', '044', '0446', '045', '047', '05', '0501', '051', '0521173159', '054', '055', '056', '0582', '059', '06', '060', '061', '062', '06319', '064', '065', '06517', '066', '06643', '067', '068', '06814', '069', '07', '070', '070

In [1]:
vectorizer = TfidfVectorizer()
vectors = vectorizer.fit_transform(val[:1000])
feature_names = vectorizer.get_feature_names()
dense = vectors.todense()
denselist = dense.tolist()
df = pd.DataFrame(denselist, columns=feature_names)

NameError: ignored

In [22]:
print(len(vectorizer.get_feature_names()))

75439


In [20]:
tfIdfTransformer = TfidfTransformer(use_idf=True)
countVectorizer = CountVectorizer()
wordCount = countVectorizer.fit_transform(val)
newTfIdf = tfIdfTransformer.fit_transform(wordCount)
df = pd.DataFrame(newTfIdf[0].T.todense(), index=countVectorizer.get_feature_names(), columns=["TF-IDF"])
df = df.sort_values('TF-IDF', ascending=False)
print (df)

               TF-IDF
abdulaziz    0.382499
jeddah       0.355092
proven       0.325661
king         0.314970
pneumoniae   0.273544
...               ...
familie      0.000000
familias     0.000000
familiarity  0.000000
familiaris   0.000000
ﬁeldwork     0.000000

[75439 rows x 1 columns]


In [11]:
lsa = TruncatedSVD(2, algorithm = 'randomized')
dtm_lsa = lsa.fit_transform(dtm)
dtm_lsa = Normalizer(copy=False).fit_transform(dtm_lsa)

In [12]:
pd.DataFrame(lsa.components_,index = ["component_1","component_2"],columns =
vectorizer.get_feature_names())

,000,0201,07,11,1101,11th,12,13,14,15,18,1918,1919,1920,1975,1985,1995,1997,1999,1beta,1h,1β,20,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,21,21st,23,2336,24,...,wheezing,whorl,wide,wild,wildlife,willebrand,willingness,winner,winter,wireless,women,woodchuck,word,work,workers,working,world,wrong,xenografts,xiao,xmrv,y271,year,years,yeast,yersinia,yields,yolk,york,young,zanamivir,zealand,zipper,zone,zoonosi,zoonosis,zoonotic,αβ,β1,κb
component_1,0.000328,0.001396,0.002729,0.000130,0.002236,0.000048,0.000284,0.000210,0.000130,0.004419,0.000397,0.019928,0.002037,0.003597,0.002289,0.000414,0.000073,0.003534,0.005095,0.000435,-2.239400e-11,0.001796,0.000283,0.000477,0.002394,0.000246,0.004508,0.003123,0.009423,0.004638,0.003569,0.210789,0.022715,0.011036,0.000001,0.000171,0.001693,0.000150,0.000543,0.000171,...,0.000374,0.000218,0.004725,0.000698,0.000224,0.000175,0.007112,0.000034,0.005276,0.000496,0.005913,0.000145,0.003336,0.002586,0.014388,0.000478,0.000372,0.000008,0.000435,0.000209,0.000348,0.002042,0.002392,0.002267,0.000934,0.000571,0.000159,0.000149,0.002289,0.002775,0.002166,0.006778,0.000085,0.000101,0.001508,0.000185,0.006917,0.000199,0.000358,0.005981
component_2,-0.000162,0.002798,-0.003372,-0.000079,0.009858,-0.000132,0.000888,0.000312,-0.000079,0.002851,0.000324,-0.014955,-0.002462,-0.005639,-0.002867,-0.000577,0.000104,0.001097,-0.002511,0.000392,-5.272011e-10,0.004557,-0.000132,0.000355,-0.002361,-0.000512,0.000904,-0.004346,-0.002234,0.002312,-0.004502,-0.162957,-0.017075,-0.007297,-0.000002,-0.000323,0.003550,-0.000283,0.001042,-0.000323,...,0.000249,0.001273,0.005435,0.001855,-0.000290,0.000767,-0.010313,-0.000044,-0.007883,0.002754,-0.006761,0.000302,-0.003450,-0.004518,-0.017884,0.000792,0.000287,0.000043,0.001444,-0.000119,0.001943,0.006780,0.006546,0.004381,0.004936,0.003061,0.000510,0.000870,-0.002867,-0.001923,-0.002649,-0.009131,0.000449,-0.000161,0.004051,0.000510,0.001979,0.000166,0.001924,0.011239


In [9]:
title = tit

In [6]:
print(title[:3])

[['clinical features culture-proven mycoplasma pneumoniae infections king abdulaziz university hospital jeddah saudi arabia'], ['nitric oxide pro-inflammatory mediator lung diseas'], ['surfactant protein-d pulmonary host defens']]


In [7]:
title = pd.DataFrame(tit)

In [4]:
import os.path
import pandas as pd
from gensim import corpora
from gensim.models import LsiModel
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from gensim.models.coherencemodel import CoherenceModel
import matplotlib.pyplot as plt

In [5]:
d = pd.read_csv("/content/COVID-19_title.csv", header=None)

In [9]:
d = d[0]

In [10]:
print(d.head())

0    clinical features culture-proven mycoplasma pneumoniae infections king abdulaziz university hospital jeddah saudi arabia
1                                                                          nitric oxide pro-inflammatory mediator lung diseas
2                                                                                  surfactant protein-d pulmonary host defens
3                                                                                               role endothelin-1 lung diseas
4                                                                gene expression epithelial cells response pneumovirus infect
Name: 0, dtype: object


In [11]:
d.dropna()

0         clinical features culture-proven mycoplasma pneumoniae infections king abdulaziz university hospital jeddah saudi arabia
1                                                                               nitric oxide pro-inflammatory mediator lung diseas
2                                                                                       surfactant protein-d pulmonary host defens
3                                                                                                    role endothelin-1 lung diseas
4                                                                     gene expression epithelial cells response pneumovirus infect
                                                                    ...                                                           
242103                                                                                        preparation organization major incid
242104                                                                             